Drop outliers

In [691]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [692]:
# Read in training set; pop sale price off of the training df to create Y_train
X_train = pd.read_csv('all/train.csv', index_col='Id')
Y_train = X_train.pop('SalePrice')

# Read in testing set
X_test = pd.read_csv('all/test.csv', index_col='Id')

# Create a combined DF to ensure that dummification works properly in test set
combined = pd.concat([X_train, X_test], axis=0, sort=False)
combined.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [693]:
# Drop unwanted features

drop_cols = ['BsmtCond', 'RoofMatl', 'MasVnrArea', 'BsmtHalfBath', 'GarageCars',
        'YearRemodAdd', 'Street', 'Alley', 'LandSlope', 'Utilities', 'Exterior2nd', 'ExterCond', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtUnfSF', 'BsmtFinType1', 'BsmtFinSF2', '1stFlrSF', '2ndFlrSF', 'KitchenAbvGr', 'FireplaceQu',
        'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PoolArea',
             'Fence', 'MiscFeature', 'MiscVal', 'SaleType']

combined = combined.drop(drop_cols, axis=1)

In [694]:
# Lot Frontage

# Build a dictionary with the median lot frontage per neighborhood from X_train
X_train_median_frontage = X_train.groupby('Neighborhood').LotFrontage.median()
X_train_median_frontage = dict(X_train_median_frontage)

# Fill in NAs by getting the corresponding value from the dictionary based on x
combined['LotFrontage'] = combined.apply(lambda x:
                                         X_train_median_frontage.get(x.Neighborhood) if pd.isna(x.LotFrontage)
                                            else x.LotFrontage, axis = 1)

In [695]:
# Define a general dummy function that either
#       1) drops the first category if all categories are meaningfully large, or
#       2) keeps the first category, but drops the small categories if any categories are not meaningfull large

# Definition of meaningfully large is taken from counts from the training data, to avoid data leakage

def standard_dummies(col):
    col_value_counts = X_train[col].value_counts()
    
    # If all of the categories are big enough... just drop the first
    if X_train[col].value_counts().min() >= 30:
        return pd.get_dummies(combined[col], drop_first=True)
    
    # If some of the categoires are quite small... drop the small ones
    else:
        all_dumms = pd.get_dummies(combined[col].astype(str))
        
        drop_dumms = col_value_counts[col_value_counts < 30].index.astype(str)
        
        all_dumms = all_dumms.drop(drop_dumms, axis = 1)
        
        all_dumms.columns = all_dumms.columns.str.slice_replace(0, 0, col + '_')
        
        return all_dumms
    
standard_dummies('MSSubClass').head()

,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_80,MSSubClass_90
Id,,,,,,,,,,,
1,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,0,0,1,0,0,0


In [696]:
# Dummify the columns that are just getting straight dummy values

# Define list of columns
std_dumm_cols = ['MSSubClass', 'Neighborhood', 'MSZoning', 'LandContour', 'LotConfig', 'KitchenQual',
                 'Exterior1st', 'HouseStyle', 'CentralAir', 'Foundation', 'GarageType', 'RoofStyle',
                 'MasVnrType', 'MoSold', 'YrSold']

# Loop through each column
for col in std_dumm_cols:
    # Get the dummies
    dummies = standard_dummies(col)
    
    # Concat to the dataframe
    combined = pd.concat([combined, dummies], axis = 1)
    
    # Drop the original column
    combined = combined.drop(col, axis = 1)

combined.head()

,LotFrontage,LotArea,LotShape,Condition1,Condition2,BldgType,OverallQual,OverallCond,YearBuilt,ExterQual,...,7,8,9,10,11,12,2007,2008,2009,2010
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,Reg,Norm,Norm,1Fam,7,5,2003,Gd,...,0,0,0,0,0,0,0,1,0,0
2,80.0,9600,Reg,Feedr,Norm,1Fam,6,8,1976,TA,...,0,0,0,0,0,0,1,0,0,0
3,68.0,11250,IR1,Norm,Norm,1Fam,7,5,2001,Gd,...,0,0,1,0,0,0,0,1,0,0
4,60.0,9550,IR1,Norm,Norm,1Fam,7,5,1915,TA,...,0,0,0,0,0,0,0,0,0,0
5,84.0,14260,IR1,Norm,Norm,1Fam,8,5,2000,Gd,...,0,0,0,0,0,1,0,1,0,0


In [697]:
# Lot Area

from math import log

combined['LotArea'] = combined['LotArea'].apply(log)

In [698]:
# Lot shape -- turn into binary for regular vs. irregular

combined['LotShape_Reg'] = combined['LotShape'].apply(lambda x: 1 if x == 'Reg' else 0)
combined = combined.drop('LotShape', axis=1)

In [699]:
# Conditions

# Identify list of conditions to check for
conditions = set(list(combined['Condition1'].unique()) + list(combined['Condition1'].unique()))
conditions.remove('Norm')

# Concatenate the two conditions for each observation
combined_conds = combined['Condition1'] + ';' + combined['Condition2']

# Loop through each condition and create a binary if it exists in that observation
for cond in conditions:
    combined['conditions_{}'.format(cond)] = 1 if cond in combined_conds else 0

# Drop the original condition field
combined = combined.drop(['Condition1', 'Condition2'], axis=1)

In [700]:
# Building type

# Group the building types -- need to group X_train as well because standard dummies fn
# relies on the value counts in X_train
combined['BldgType'] = ['Twnhs' if x == 'TwnhsE' else '2fam' if x in ['Duplex', '2fmCon'] else x
                        for x in combined['BldgType']]
X_train['BldgType'] = ['Twnhs' if x == 'TwnhsE' else '2fam' if x in ['Duplex', '2fmCon'] else x
                        for x in X_train['BldgType']]

# Dummify and drop original column
combined = pd.concat([combined, standard_dummies('BldgType')], axis = 1)
combined = combined.drop('BldgType', axis=1)

In [701]:
# Overall Condition -- binary for if it is high quality (greater than or equal to 7)

combined['OverallCond'] = combined['OverallCond'].apply(lambda x: 1 if x >=7 else 0)

In [702]:
# Year Built -- enforce a minimum value of 1950

combined['YearBuilt'] = combined['YearBuilt'].apply(lambda x: 1950 if x < 1950 else x)

In [703]:
# Exterior Quality

# Combine Typical and Fair
combined['ExterQual'] = combined['ExterQual'].apply(lambda x: 'TA' if x == 'Fa' else x)
X_train['ExterQual'] = X_train['ExterQual'].apply(lambda x: 'TA' if x == 'Fa' else x)

# Dummify and drop original column
combined = pd.concat([combined, standard_dummies('ExterQual')], axis = 1)
combined = combined.drop('ExterQual', axis=1)

In [704]:
# Basement Quality... same thing

# Combine Typical and Fair
combined['BsmtQual'] = combined['BsmtQual'].apply(lambda x: 'TA' if x == 'Fa' else x)
X_train['BsmtQual'] = X_train['BsmtQual'].apply(lambda x: 'TA' if x == 'Fa' else x)

# Dummify and drop original column
combined = pd.concat([combined, standard_dummies('BsmtQual')], axis = 1)
combined = combined.drop('BsmtQual', axis=1)

In [705]:
# Basement Exposure -- is it avg or good?

combined['BsmtExposure_pos'] = combined['BsmtExposure'].apply(lambda x: 1 if x in ['Av', 'Gd'] else 0)

combined = combined.drop('BsmtExposure', axis = 1)

In [706]:
# Heating Type -- is it GasA?

combined['Heating_GasA'] = combined['Heating'].apply(lambda x: 1 if x == 'GasA' else 0)

combined = combined.drop('Heating', axis = 1)

In [707]:
# Heating Quality -- is it excellent?

combined['HeatingQC_ex'] = combined['HeatingQC'].apply(lambda x: 1 if x == 'Ex' else 0)

combined = combined.drop('HeatingQC', axis=1)

In [708]:
# Electrical -- breaker?

combined['Electrical_brkr'] = combined['Electrical'].apply(lambda x: 1 if x == 'SBrkr' else 0)

combined = combined.drop('Electrical', axis = 1)

In [709]:
# Low Qual Fin Sq Ft -- is there any?

combined['LowQualFinSF_y'] = combined['LowQualFinSF'].apply(lambda x: 1 if x > 0 else 0)

combined = combined.drop('LowQualFinSF', axis = 1)

In [710]:
# Half Bath -- is there one?

combined['HalfBath_y'] = combined['HalfBath'].apply(lambda x: 1 if x > 0 else 0)

combined = combined.drop('HalfBath', axis=1)

In [711]:
# Functional -- is there any damage?

combined['Functional_dmg'] = combined['Functional'].apply(lambda x: 0 if x == 'Typ' else 1)

combined = combined.drop('Functional', axis = 1)

In [712]:
# Fireplaces -- are they any?

combined['Fireplaces'] = combined['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [713]:
# Paved Drive -- is there a good paved driveway?

combined['PavedDrive'] = combined['PavedDrive'].apply(lambda x: 1 if x == 'Y' else 0)

In [714]:
# Porches -- what is the size and type of the largest porch?

# Gets a series of the porch sizes for each record;
# Sort the series by size; and returns idx(type)/value as a tuple
combined['Porch_tuple'] = combined.filter(regex = 'Porch|Deck').apply(lambda s:
                                                    list(s.sort_values(ascending=False)[[0]].iteritems())[0],
                                                                     axis = 1)

# Extract the size and type as separate columns from the tuple
combined['Porch_largest_type'] = combined['Porch_tuple'].apply(lambda t: t[0])
combined['porch_largest_size'] = combined.pop('Porch_tuple').apply(lambda t: t[1])

# Convert porch type to no porch if biggest size is zero
combined['Porch_largest_type'] = combined.apply(lambda s: 'No Porch' if s.porch_largest_size == 0
                                                else s.Porch_largest_type, axis=1)
                                                
# Get dummies for the largest porch type
combined = pd.concat([combined, pd.get_dummies(combined['Porch_largest_type'], drop_first=True)], axis=1)

# Drop the original columns and the tuple
combined = combined.drop(combined.filter(regex = 'Porch|Deck').columns, axis=1)

In [715]:
# Pool -- is there one?

combined['Pool_y'] = combined['PoolQC'].apply(lambda x: 0 if pd.isna(x) else 1)

combined = combined.drop('PoolQC', axis = 1)

In [716]:
# Sale condition -- is it partial or abnormal?

# Flag partial and abnormal sales
combined['SaleCondition_Partial'] = combined['SaleCondition'].apply(lambda x: 1 if x == 'Partial' else 0)
combined['SaleCondition_Abnorml'] = combined['SaleCondition'].apply(lambda x: 1 if x == 'Abnorml' else 0)

# Drop the original column
combined = combined.drop('SaleCondition', axis=1)

In [717]:
# Check dataframe

combined.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,GrLivArea,BsmtFullBath,FullBath,BedroomAbvGr,...,Heating_GasA,HeatingQC_ex,Electrical_brkr,LowQualFinSF_y,HalfBath_y,Functional_dmg,porch_largest_size,Pool_y,SaleCondition_Partial,SaleCondition_Abnorml
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,9.041922,7,0,2003,856.0,1710,1.0,2,3,...,1,1,1,0,1,0,61,0,0,0
2,80.0,9.169518,6,1,1976,1262.0,1262,0.0,2,3,...,1,1,1,0,0,0,298,0,0,0
3,68.0,9.328123,7,0,2001,920.0,1786,1.0,2,3,...,1,1,1,0,1,0,42,0,0,0
4,60.0,9.164296,7,0,1950,756.0,1717,1.0,1,3,...,1,0,1,0,0,0,272,0,0,1
5,84.0,9.565214,8,0,2000,1145.0,2198,1.0,2,4,...,1,1,1,0,1,0,192,0,0,0


In [718]:
# Confirm that we only have numerical values left

combined.dtypes.value_counts()

uint8      89
int64      29
float64     5
dtype: int64

In [722]:
# Check for any remaining NAs

combined.apply(lambda col: col.isna().sum()).sort_values(ascending=False).head()

BsmtFullBath            2
GarageArea              1
TotalBsmtSF             1
Neighborhood_SawyerW    0
MSZoning_RM             0
dtype: int64

In [723]:
# Fill remaining NAs with 0s

combined = combined.fillna(0)

In [724]:
# Split combined df back into X train and X test based on the index values

X_train = combined.loc[X_train.index, :]
X_test = combined.loc[X_test.index, :]

print('X_train: {} obvs\nX_test: {} obvs'.format(len(X_train), len(X_test)))

X_train: 1460 obvs
X_test: 1459 obvs


In [727]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train, Y_train)
lm.score(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

0.8732179609792378

In [728]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, Y_train)
rf.score(X_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

0.9567413029676661